# Cognitive Cost of Quantization — Colab Setup

This notebook sets up an SSH tunnel into Google Colab so you can run Claude Code
and benchmarks directly on the T4 GPU.

**Steps:**
1. Run Cell 1 to install SSH + cloudflared
2. Run Cell 2 to configure SSH server
3. Run Cell 3 to start the tunnel — copy the SSH command
4. SSH from your Mac, then run `./setup_colab.sh`

**Required Colab Secrets:**
- `HF_TOKEN` — HuggingFace access token
- `ANTHROPIC_AUTH_TOKEN` — Mercury auth token for Claude Code
- `OPENAI_API_KEY` — OpenAI API key for Codex CLI

In [ ]:
# Cell 1: Install cloudflared + openssh-server
!apt-get update -qq && apt-get install -qq -y openssh-server > /dev/null 2>&1
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1
!rm cloudflared-linux-amd64.deb
print('cloudflared + openssh installed')

In [ ]:
# Cell 2: Configure SSH server
import subprocess

# Set root password
PASSWORD = 'colab-research-2024'  # Change if desired
subprocess.run(['bash', '-c', f'echo "root:{PASSWORD}" | chpasswd'], check=True)

# Configure sshd to allow root login + password auth
sshd_config = """
PermitRootLogin yes
PasswordAuthentication yes
ChallengeResponseAuthentication no
UsePAM yes
"""
with open('/etc/ssh/sshd_config', 'a') as f:
    f.write(sshd_config)

# Generate host keys if missing and start sshd
subprocess.run(['ssh-keygen', '-A'], check=True, capture_output=True)
subprocess.run(['service', 'ssh', 'start'], check=True)

# Export Colab Secrets as environment variables for setup_colab.sh
from google.colab import userdata
import os

secrets = ['HF_TOKEN', 'ANTHROPIC_AUTH_TOKEN', 'OPENAI_API_KEY']
env_lines = []
for secret in secrets:
    try:
        val = userdata.get(secret)
        os.environ[secret] = val
        env_lines.append(f'export {secret}="{val}"')
        print(f'{secret}: set')
    except Exception:
        print(f'{secret}: NOT SET (add to Colab Secrets)')

# Write to /etc/environment so SSH sessions get them
with open('/etc/environment', 'a') as f:
    for line in env_lines:
        f.write(line + '\n')

# Also write a .env file root can source
with open('/root/.bashrc_secrets', 'w') as f:
    for line in env_lines:
        f.write(line + '\n')

# Append source to .bashrc so SSH sessions pick up secrets
with open('/root/.bashrc', 'a') as f:
    f.write('\n# Colab secrets\n')
    f.write('[ -f ~/.bashrc_secrets ] && source ~/.bashrc_secrets\n')

print(f'\nSSH server started. Password: {PASSWORD}')

In [ ]:
# Cell 3: Start cloudflared tunnel and print SSH command
import subprocess
import re
import time
import threading

# Start cloudflared in background
proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'ssh://localhost:22', '--no-autoupdate'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Read stderr in a thread to find the tunnel URL
tunnel_url = None
lines = []

def read_output():
    global tunnel_url
    for line in proc.stderr:
        lines.append(line.strip())
        match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
        if match:
            tunnel_url = match.group(0)

thread = threading.Thread(target=read_output, daemon=True)
thread.start()

# Wait for URL
for _ in range(30):
    if tunnel_url:
        break
    time.sleep(1)

if tunnel_url:
    hostname = tunnel_url.replace('https://', '')
    print('=' * 60)
    print('SSH TUNNEL READY')
    print('=' * 60)
    print()
    print('From your Mac terminal, run:')
    print()
    print(f'ssh -o ProxyCommand="cloudflared access ssh --hostname {hostname}" root@{hostname}')
    print()
    print(f'Password: colab-research-2024')
    print()
    print('Then on the Colab machine:')
    print('  cd ~/cognitive-cost-quantization && ./setup_colab.sh')
    print('=' * 60)
else:
    print('ERROR: Tunnel URL not found after 30s')
    print('Last lines:', lines[-5:])

In [ ]:
# Cell 4: Keep-alive (run this to prevent Colab from disconnecting)
import time
import IPython.display

print('Keep-alive running. Do NOT stop this cell.')
print('Your SSH session will stay active as long as this cell runs.')
print()

counter = 0
while True:
    counter += 1
    IPython.display.clear_output(wait=True)
    print(f'Keep-alive: {counter} min elapsed. SSH tunnel active.')
    print(f'GPU: ', end='')
    !nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,noheader
    time.sleep(60)